신경망 모델 구성하기

In [6]:
import os #dir 파일 경로 확인
import torch 
from torch import nn # nn구성 함수가 들어있는 라이브러리
from torch.utils.data import DataLoader 
from torchvision import datasets, transforms 


학습을 위한 장치 얻기

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


클래스 정의하기

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        # sequential : 연산 여러개를 한꺼번에 -> 순차적으로. 
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NeuralNetwork 인스턴스 생성

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
model.linear_relu_stack[2].weight

Parameter containing:
tensor([[-0.0093, -0.0192, -0.0436,  ..., -0.0309,  0.0049, -0.0173],
        [ 0.0308, -0.0388,  0.0223,  ..., -0.0384, -0.0067, -0.0225],
        [-0.0029,  0.0354,  0.0136,  ..., -0.0046, -0.0165, -0.0157],
        ...,
        [-0.0253, -0.0278,  0.0052,  ..., -0.0390, -0.0310,  0.0253],
        [ 0.0436,  0.0025, -0.0069,  ..., -0.0265, -0.0349,  0.0317],
        [-0.0310,  0.0421,  0.0227,  ..., -0.0020,  0.0308, -0.0126]],
       device='cuda:0', requires_grad=True)

모델을 사용하기 위해 입력 데이터 전달

In [5]:
# 데이터입력값
X = torch.rand(1, 28, 28, device = device)
# 원시 예측값 (2차원 텐서 반환. dim=0: 각 class에 대한 raw_pred 10개, dim=1: 각 출력의 개별 값)
logits = model(X)
# 예측 확률
# 원시 예측값을 소프트맥스 함수를 사용하여 확률로 변환.  dim=1은 각 샘플에 대해 소프트맥스 함수를 적용하고자 하는 축을 지정
#dim=0이면 각 열을 기준으로 소프트맥스가 계산되고, dim=1이면 각 행을 기준으로 소프트맥스가 계산.
pred_probab = nn.Softmax(dim = 1)(logits)
# argmax는 가장 큰 값을 갖는 텐서의 인덱스를 출력하는 함수. 여기서 1은 기준 축을 의미.
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


모델 계층 (Layer)

In [9]:
# 28*28 크기의 이미지 3개로 구성된 미니배치 가져오기
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten

In [10]:
flatten = nn.Flatten()
# nn.Flatten 계층을 초기화해, 28*28의 2D 이미지를 784픽셀의 연속된 배열로 변환
# dim=0 의 미니배치차원(torch.Size([3, 784]에서 3을 의미)은 유지    
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
> 저장된 가중치 (weight)와 편향(bias)을 사용해 입력에 선형변환을 적용

In [11]:
layer1= nn.Linear(in_features=28*28, out_features =20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU

> 음수는 0으로 대체됨

In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2896, -0.2576, -0.5705, -0.0360, -0.2172, -0.2067, -0.0269,  0.3022,
          0.5741,  0.4558,  0.0926, -0.6318,  0.6971, -0.3391, -0.0194, -0.2867,
          0.2350, -0.2284,  0.3050, -0.3578],
        [ 0.2483,  0.0979, -0.7292, -0.1237, -0.0335,  0.0503,  0.2763,  0.5774,
          0.1041,  0.2437,  0.4904, -0.4468,  0.4251, -0.4840, -0.4445, -0.5006,
          0.1947, -0.2053,  0.6312, -0.5017],
        [ 0.5822,  0.2414, -0.5770, -0.0181,  0.1839,  0.1270,  0.3097, -0.0236,
         -0.0910,  0.1575, -0.0762, -0.5402,  0.5960, -0.1332, -0.0810, -0.2235,
          0.0468, -0.3239,  0.5517, -0.2793]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2896, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3022, 0.5741,
         0.4558, 0.0926, 0.0000, 0.6971, 0.0000, 0.0000, 0.0000, 0.2350, 0.0000,
         0.3050, 0.0000],
        [0.2483, 0.0979, 0.0000, 0.0000, 0.0000, 0.0503, 0.2763, 0.5774, 0.1041,
         0.2437, 0.4904, 0.0000, 0.4251, 0.0000, 0.00

nn.Sequential
> 순서를 갖는 모듈의 컨테이너

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image) # logits 값은 [-infty, infty] 범위.

nn.Softmax

> logits는 모델의 각 class에 대한 예측 확률을 나타내도록 [0,1] 범위로 비례하여 조정됨. dim 매개변수는 값의 합이 1이 되는 차원을 나타냄.

In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

모델 매개변수

In [17]:
print(f"Model structure: {model}\n\n")
# nn.moule을 상속하면, 모델 객체 내부의 모든 필드들이 자동으로 추적됨. parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근 가능.
# 각 매개변수들마다 매개변수의 크기와 값 출력
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0239,  0.0170, -0.0193,  ..., -0.0325,  0.0263, -0.0120],
        [-0.0329, -0.0105, -0.0104,  ..., -0.0248,  0.0233, -0.0348]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0209,  0.0047], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0271, -0.0153, -0.0226,  ...,  0.0021, -0.0315, -0.0240],
        [-0.0398,  0.0101, -0.0177,  ..., -0.0317,  0.0219, -0.0017]],
       device='cuda:0', grad_fn=<Slice